In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import re
import requests
import json


In [7]:
vespa_df = pd.read_csv("gpcr_vespa_df.csv")
vespa_df

,Uniprot_ID,protein,VESPAl,mutation_from,mutation_pos,mutation_to
0,Q13639,5ht4r_human,0.587059,Y,302,A
1,Q13639,5ht4r_human,0.366272,Y,302,F
2,Q13639,5ht4r_human,0.592905,N,279,L
3,Q13639,5ht4r_human,0.554260,F,276,V
4,Q13639,5ht4r_human,0.581045,F,275,A
...,...,...,...,...,...,...
2480,Q03431,pth1r_human,0.325055,V,239,A
2481,Q03431,pth1r_human,0.506099,K,240,A
2482,P41587,vipr2_human,0.166512,Y,184,F
2483,P41587,vipr2_human,0.401903,K,179,Q


In [3]:
# GPCRdb data
GPCRdb_fil = pd.read_csv("gpcr_filtered_mutants_0217.csv")
GPCRdb_fil


,Unnamed: 0,reference,protein,mutation_pos,mutation_from,mutation_to,ligand_name,ligand_idtype,ligand_id,ligand_class,...,exp_mu_effect_value,exp_fold_change,exp_mu_effect_qual,exp_mu_effect_ligand_prop,exp_mu_ligand_ref,opt_receptor_expression,opt_basal_activity,opt_gain_of_activity,opt_ligand_emax,opt_agonist
0,22,15626750,hrh1_human,37,I,V,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,7.700,-1.585,NaN,NaN,[3H]-mepyramine (radioligand),94.2,0.0,NaN,0.0,NaN
1,23,15626750,hrh1_human,38,C,S,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,7.900,1.000,NaN,NaN,[3H]-mepyramine (radioligand),141.3,0.0,NaN,0.0,NaN
2,24,15626750,hrh1_human,84,N,S,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,7.500,-2.513,NaN,NaN,[3H]-mepyramine (radioligand),53.7,0.0,NaN,0.0,NaN
3,26,16408006,hrh1_human,111,S,A,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,7.300,1.000,NaN,NaN,[3H]-mepyramine (radioligand),58.3,0.0,NaN,0.0,NaN
4,27,16408006,hrh1_human,111,S,C,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,7.500,1.585,NaN,NaN,[3H]-mepyramine (radioligand),55.0,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11974,455,10.1124/mol.63.5.1021,aa3r_human,274,N,A,VUF-5455,PubChem CID,10339071,Allosteric inverse agonist,...,0.083,-3.322,NaN,NaN,I-AB-MECA,0.0,0.0,=,0.0,NaN
11975,456,10.1124/mol.63.5.1021,aa3r_human,274,N,A,I-AB-MECA,PubChem CID,44208896,Full agonist,...,0.085,-1.328,NaN,NaN,NaN,0.0,0.0,=,0.0,NaN
11976,457,10.1006/bbrc.2001.5027,aa3r_human,282,Y,F,Cl-IB-MECA,PubChem CID,393593,Full agonist,...,27.600,-29.412,NaN,NaN,NaN,0.0,0.0,=,0.0,NaN
11977,458,10.1006/bbrc.2001.5027,aa3r_human,282,Y,F,I-AB-MECA,PubChem CID,44208896,Full agonist,...,9.300,-7.752,NaN,NaN,NaN,0.0,0.0,=,0.0,NaN


In [4]:
# merge two dataframes
gpcr_vespa  = pd.merge(GPCRdb_fil,vespa_df,on=['mutation_pos','mutation_from','mutation_to','protein'])
gpcr_vespa

,Unnamed: 0,reference,protein,mutation_pos,mutation_from,mutation_to,ligand_name,ligand_idtype,ligand_id,ligand_class,...,exp_mu_effect_qual,exp_mu_effect_ligand_prop,exp_mu_ligand_ref,opt_receptor_expression,opt_basal_activity,opt_gain_of_activity,opt_ligand_emax,opt_agonist,Uniprot_ID,VESPAl
0,22,15626750,hrh1_human,37,I,V,Cetirizine,ChEMBL Compound ID,CHEMBL1000,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),94.2,0.0,NaN,0.0,NaN,P35367,0.353884
1,78,15626750,hrh1_human,37,I,V,HP-HA,ChEMBL Compound ID,CHEMBL264491,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),94.2,0.0,NaN,0.0,NaN,P35367,0.353884
2,106,15626750,hrh1_human,37,I,V,[3H]-mepyramine,ChEMBL Compound ID,CHEMBL511,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),94.2,0.0,NaN,0.0,NaN,P35367,0.353884
3,164,15626750,hrh1_human,37,I,V,Histamine,ChEMBL Compound ID,CHEMBL90,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),94.2,0.0,NaN,0.0,NaN,P35367,0.353884
4,210,15626750,hrh1_human,37,I,V,HP-HP,SMILES,C1(CCNCCC2=CNC(CCC(C3=CC=CC=C3)C4=CC=CC=C4)=N2...,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),94.2,0.0,NaN,0.0,NaN,P35367,0.353884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9704,437,10.1021/jm050968b,aa3r_human,272,H,D,CID75022483,PubChem CID,75022483,Full agonist,...,NaN,NaN,I-AB-MECA,0.0,0.0,=,0.0,NaN,P0DMS8,0.563917
9705,439,10.1021/jm050968b,aa3r_human,272,H,D,CHEMBL377071,ChEMBL Compound ID,377071,Full agonist,...,NaN,NaN,I-AB-MECA,0.0,0.0,=,0.0,NaN,P0DMS8,0.563917
9706,457,10.1006/bbrc.2001.5027,aa3r_human,282,Y,F,Cl-IB-MECA,PubChem CID,393593,Full agonist,...,NaN,NaN,NaN,0.0,0.0,=,0.0,NaN,P0DMS8,0.675531
9707,458,10.1006/bbrc.2001.5027,aa3r_human,282,Y,F,I-AB-MECA,PubChem CID,44208896,Full agonist,...,NaN,NaN,NaN,0.0,0.0,=,0.0,NaN,P0DMS8,0.675531


after merging, we lost one combination since we exclude DRD4 (P21917)mutation which happened at position 430 but in original only length 419

In [5]:
gpcr_vespa.to_csv('GPCR_VESPAI_df.csv',index=False)

## classification performance

In [14]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, roc_curve, auc, confusion_matrix

In [19]:
gpcr_vespa['VESPAl']

,Unnamed: 0,reference,protein,mutation_pos,mutation_from,mutation_to,ligand_name,ligand_idtype,ligand_id,ligand_class,...,exp_mu_effect_qual,exp_mu_effect_ligand_prop,exp_mu_ligand_ref,opt_receptor_expression,opt_basal_activity,opt_gain_of_activity,opt_ligand_emax,opt_agonist,Uniprot_ID,VESPAl
0,0,8903934,hrh1_human,194,T,A,(S)-cetirizine,ChEMBL Compound ID,CHEMBL1334217,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),0.0,0.0,NaN,0.0,NaN,P35367,0.450196
1,8,8198587,hrh1_human,194,T,A,2-methylhistamine,ChEMBL Compound ID,CHEMBL12620,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),126.0,0.0,NaN,0.0,NaN,P35367,0.450196
2,12,11809864,hrh1_human,194,T,A,(S)-cetirizine,ChEMBL Compound ID,CHEMBL1334217,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),24.4,0.0,NaN,0.0,NaN,P35367,0.450196
3,19,8093027,hrh1_human,194,T,A,[3H]-mepyramine,ChEMBL Compound ID,CHEMBL511,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),122.7,0.0,NaN,0.0,NaN,P35367,0.450196
4,20,8198587,hrh1_human,194,T,A,Histamine,ChEMBL Compound ID,CHEMBL90,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),126.0,0.0,NaN,0.0,NaN,P35367,0.450196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12016,438,10.1006/bbrc.2001.5027,aa3r_human,107,D,R,I-AB-MECA,PubChem CID,44208896,Full agonist,...,NaN,NaN,NaN,0.0,0.0,=,0.0,NaN,P0DMS8,0.757770
12017,233,10.1021/jm050968b,aa3r_human,94,T,E,CID85116573,PubChem CID,85116573,Full agonist,...,NaN,NaN,I-AB-MECA,0.0,0.0,=,0.0,NaN,P0DMS8,0.546532
12018,271,10.1021/jm050968b,aa3r_human,94,T,E,CHEMBL377071,ChEMBL Compound ID,377071,Full agonist,...,NaN,NaN,I-AB-MECA,0.0,0.0,=,0.0,NaN,P0DMS8,0.546532
12019,279,10.1021/jm050968b,aa3r_human,94,T,E,CID75022483,PubChem CID,75022483,Full agonist,...,NaN,NaN,I-AB-MECA,0.0,0.0,=,0.0,NaN,P0DMS8,0.546532


In [35]:
# VESPAl into two classifications: >=0.5(positive) <0.5(negative)
gpcr_vespa['VESPAl_group'] = gpcr_vespa['VESPAl'].apply(lambda x: 1 if x >= 0.5 else 0)
gpcr_vespa['foldchange_group'] = gpcr_vespa['exp_fold_change'].apply(lambda x: 1 if x <= -5 else (1 if x >= 5 else 0))


In [36]:
true_labels = gpcr_vespa['foldchange_group']
predicted_labels = gpcr_vespa['VESPAl_group']

In [37]:
# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(true_labels, predicted_labels).ravel()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate specificity and sensitivity
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)


# Calculate precision, recall and F1-score
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# calculate FPR and TPR
fpr, tpr, thresholds = roc_curve(true_labels, predicted_labels)

# calculate AUCROC
auc_roc = auc(fpr, tpr)



print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("AUCROC score: {:.3f}".format(auc_roc))

Specificity: 0.6598351001177857
Sensitivity: 0.4868309260832625
Accuracy: 0.6090175526162549
Precision: 0.37312784892554807
F1-score: 0.422462521504055
AUCROC score: 0.573


In [24]:
# Generate classification report
report = classification_report(true_labels, predicted_labels, labels=[0, 1], digits=4,zero_division="warn")

# Print report
print(report)


              precision    recall  f1-score   support

           0     0.8501    0.6462    0.7343      9754
           1     0.2509    0.5099    0.3363      2267

    accuracy                         0.6205     12021
   macro avg     0.5505    0.5781    0.5353     12021
weighted avg     0.7371    0.6205    0.6592     12021

